# Chatbot

## Imports

In [3]:
import numpy as np
import nltk
from nltk.stem.porter import PorterStemmer
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import re
from datasets import load_dataset
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer
from transformers import EncoderDecoderModel
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from sacrebleu import corpus_bleu
import pandas as pd
import numpy as np
import pyarabic.araby as araby
import pyarabic.number as number
import pyarabic.named as named
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## Loading Dataset

In [4]:
df_total=pd.read_csv("totalDatasetWithEmo2.csv",encoding="utf-8-sig")
df_totalTokens=pd.read_csv("totalDatasetTokensWithEmo2.csv",encoding="utf-8-sig")

## Initializing emotion-->response and emotion-->contet dictionary for easy retrieval

In [5]:
from collections import defaultdict
# set(trainData1['emotion'].to_list())
d=defaultdict(list)
d2=defaultdict(list)
d3=defaultdict(list)
d4=defaultdict(list)
for i in range(df_total.shape[0]):
    d[df_total['emotion'][i]].append(df_total["response"][i])
    d2[df_total['emotion'][i]].append(df_total["context"][i])
    d3[df_total['emotion2'][i]].append(df_total["response"][i])
    d4[df_total['emotion2'][i]].append(df_total["context"][i])

## handling arabic text when the user inputs it

In [6]:
lista=re.compile(
    u"([^\u0621-\u06D3\ufb50-\ufdff\ufe70-\ufeff\u0750-\u077f])", re.UNICODE)
def is_arabicrange(text):
    """ Checks for an  Arabic Unicode block characters
    @param text: input text
    @type text: unicode
    @return: True if all charaters are in Arabic block
    @rtype: Boolean
    """
    if lista.search(text):
        return False
    return True

## Simple IR chatbot without using model for comparison (slowest)

In [7]:

#import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 

colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

# with open("intents.json") as file:
#     data = json.load(file)


def chat():
    # load trained model
#     model = keras.models.load_model('chat_model')

#     # load tokenizer object
#     with open('tokenizer.pickle', 'rb') as handle:
#         tokenizer = pickle.load(handle)

#     # load label encoder object
#     with open('label_encoder.pickle', 'rb') as enc:
#         lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        #result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),truncating='post', maxlen=max_len))
        #tag = lbl_encoder.inverse_transform([np.argmax(result)])
        similarity=[]
        for i in range(df_total.shape[0]):
#             if i == tag:
#             listOfLists=dresponse[i]
#             similarity=[]
#             for j in range(len(listOfLists)):
            l1 =[]
            l2 =[]
            inputvalue=araby.tokenize(inp, conditions=is_arabicrange, morphs=araby.strip_tashkeel)
            context=araby.tokenize(df_total["context"][i], conditions=is_arabicrange, morphs=araby.strip_tashkeel)
            inp_set = {w for w in inputvalue} 
            context_set = {w for w in context}
            rvector = inp_set.union(context_set) 
            for w in rvector:
                if w in inp_set: l1.append(1) # create a vector
                else: l1.append(0)
                if w in context_set: l2.append(1)
                else: l2.append(0)
            c = 0
 
            for j in range(len(rvector)):
                c+= l1[j]*l2[j]
            if(float((sum(l1)*sum(l2))**0.5)!=0):
                cosine = c / float((sum(l1)*sum(l2))**0.5)
            else: cosine=0
            similarity.append(cosine)
        x=np.argmax(similarity)
        print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , df_total["response"][x])

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: أهلا بك
ChatBot: أهلا 
User: stop
ChatBot: هل كان هذا صديقا كنت تحبه أم مجرد أفضل صديق؟
User: quit


KeyboardInterrupt: 

## Chatbot using Deep Learning Model classifying sentiment (Quickest)

In [8]:

#import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 

colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

# with open("intents.json") as file:
#     data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model5')

    # load tokenizer object
    with open('tokenizer5.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder5.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 25
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break
        
        inp2=araby.tokenize(inp, conditions=is_arabicrange, morphs=araby.strip_tashkeel)
        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp2]),
                                             truncating='post', maxlen=max_len))
#         print(result)
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in d3:
            if i == tag:
                print(tag)
                listOfLists=d4[i]
                similarity=[]
                for j in range(len(listOfLists)):
                    l1 =[]
                    l2 =[]
                    inputvalue=araby.tokenize(inp, conditions=is_arabicrange, morphs=araby.strip_tashkeel)
                    context=araby.tokenize(listOfLists[j], conditions=is_arabicrange, morphs=araby.strip_tashkeel)
                    
                    
                    
                    inp_set = {w for w in inputvalue} 
                    context_set = {w for w in context}
                    rvector = inp_set.union(context_set) 
                    for w in rvector:
                        if w in inp_set: l1.append(1) # create a vector
                        else: l1.append(0)
                        if w in context_set: l2.append(1)
                        else: l2.append(0)
                    c = 0
 
                    for j in range(len(rvector)):
                        c+= l1[j]*l2[j]
                    if(float((sum(l1)*sum(l2))**0.5)!=0):
                        cosine = c / float((sum(l1)*sum(l2))**0.5)
                    else: cosine=0
                    similarity.append(cosine)
#                     similarity.append(c)
                x=np.argmax(similarity)
#                 print(similarity)
#                 print(d4[i][x])
#                 print(x)
                listOfSimilars=[]
#                 listOfSimilarscontext=[]
                if np.amax(similarity)==0:
                    print("انا لم افهم ما تقول")
                else:
                    for k in range(len(similarity)):
                        if similarity[k]==similarity[x]:
                            listOfSimilars.append(d3[i][k])
#                             listOfSimilarscontext.append(d4[i][k])
#                     print(listOfSimilarscontext)
                    print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , random.choice(listOfSimilars))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: أهلا بك
['happy']
ChatBot: أهلا 
User: أشعر بالألم
['sad']
ChatBot: أوه لا لماذا؟
User: quit


In [ ]:
# Choosing the first maximum answer (quickest)

# #import json 
# import numpy as np
# from tensorflow import keras
# from sklearn.preprocessing import LabelEncoder

# import colorama 

# colorama.init()
# from colorama import Fore, Style, Back

# import random
# import pickle

# # with open("intents.json") as file:
# #     data = json.load(file)


# def chat():
#     # load trained model
#     model = keras.models.load_model('chat_model5')

#     # load tokenizer object
#     with open('tokenizer5.pickle', 'rb') as handle:
#         tokenizer = pickle.load(handle)

#     # load label encoder object
#     with open('label_encoder5.pickle', 'rb') as enc:
#         lbl_encoder = pickle.load(enc)

#     # parameters
#     max_len = 25
    
#     while True:
#         print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
#         inp = input()
#         if inp.lower() == "quit":
#             break
        
#         inp2=araby.tokenize(inp, conditions=is_arabicrange, morphs=araby.strip_tashkeel)
#         result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp2]),
#                                              truncating='post', maxlen=max_len))
# #         print(result)
#         tag = lbl_encoder.inverse_transform([np.argmax(result)])

#         for i in d3:
#             if i == tag:
# #                 print(tag)
#                 listOfLists=d4[i]
#                 similarity=[]
#                 for j in range(len(listOfLists)):
#                     l1 =[]
#                     l2 =[]
#                     inputvalue=araby.tokenize(inp, conditions=is_arabicrange, morphs=araby.strip_tashkeel)
#                     context=araby.tokenize(listOfLists[j], conditions=is_arabicrange, morphs=araby.strip_tashkeel)
                    
                    
                    
#                     inp_set = {w for w in inputvalue} 
#                     context_set = {w for w in context}
#                     rvector = inp_set.union(context_set) 
#                     for w in rvector:
#                         if w in inp_set: l1.append(1) # create a vector
#                         else: l1.append(0)
#                         if w in context_set: l2.append(1)
#                         else: l2.append(0)
#                     c = 0
 
#                     for j in range(len(rvector)):
#                         c+= l1[j]*l2[j]
#                     if(float((sum(l1)*sum(l2))**0.5)!=0):
#                         cosine = c / float((sum(l1)*sum(l2))**0.5)
#                     else: cosine=0
#                     similarity.append(cosine)
#                 x=np.argmax(similarity)
# #                 print(similarity)
# #                 print(d4[i][x])
# #                 print(x)
#                 if np.amax(similarity)==0:
#                     print("انا لم افهم ما تقول")
#                 else:
#                     print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , d3[i][x])

#         # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

# print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
# chat()